In [ ]:
from sklearn import svm
import dalex
import pandas as pd
from category_encoders import TargetEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import Normalizer
from sklearn.metrics import mean_squared_error,  make_scorer
import numpy as np

In [ ]:
apartments=dalex.datasets.load_apartments()
apartments

In [ ]:
cereal=pd.read_csv("cereal.csv")
print(cereal)
# Ze względu, że SVC wymaga liczb całkowitych jako target zastąpię rating podłogą z wartości kolumny
cereal["rating"]=cereal["rating"].round(0).astype(int)

## Preprocessing

In [ ]:
te=TargetEncoder()
apartments['district']=te.fit_transform(apartments['district'],apartments['m2_price'])

y=apartments['m2_price']
X=apartments.iloc[:,1:]
trainx,testx,trainy,testy=train_test_split(X,y,test_size=0.3,random_state=23)

In [ ]:
te=TargetEncoder()
cereal['mfr']=te.fit_transform(cereal['mfr'],cereal['rating'])



In [ ]:
te=TargetEncoder()
cereal['type']=te.fit_transform(cereal['type'],cereal['rating'])

# Apartments

## Standard parameters without standarisation

In [ ]:
svc=svm.SVC()
svc.fit(trainx,trainy)
ans=svc.predict(testx)
print(f"Mean squared error:  { mean_squared_error(ans,testy, squared=False)}")

## Standard parameters with normalisation

In [ ]:
scaler=Normalizer().fit(apartments.iloc[:,1:])
apartments2=pd.DataFrame(scaler.transform(apartments.iloc[:,1:]))
y=apartments['m2_price']
X=apartments2.iloc[:,1:]
trainx2,testx2,trainy2,testy2=train_test_split(X,y,test_size=0.3,random_state=2123)
svc2=svm.SVC()
svc2.fit(trainx2,trainy2)
ans2=svc2.predict(testx2)
print(f"Mean squared error:  { mean_squared_error(ans2,testy2, squared=False)}")


Jak widać dzięki standaryzacji udało się osiągnąć lepszy wynik

## Hyperparameters Random Search

In [ ]:
model_params={
    "kernel":["rbf","poly"],
    "gamma":['scale','auto'],
    "degree":[2,3,4,5,6]
    
}

In [ ]:
rs=RandomizedSearchCV(estimator = svm.SVC(),param_distributions = model_params, cv=3,n_jobs=-1,n_iter=100, random_state=1613,verbose=1,scoring='neg_root_mean_squared_error')
rs.fit(trainx2,trainy2)

In [ ]:
print(f"Mean squared error:  { rs.best_score_}")
print(f"Hyperparameters:  { rs.best_params_}")

Jak widać dostrojenie hiperparametrów dość mocno wpłynęło na poprawę wyniku.

# Cereals

Baza danych pobrana z: https://www.kaggle.com/crawford/80-cereals , w której targetem jest rating

In [ ]:
y=cereal['rating']
X=cereal.iloc[:,1:-1]
trainx,testx,trainy,testy=train_test_split(X,y,test_size=0.3,random_state=23)

## Standard parameters without standarisation

In [ ]:
svc=svm.SVC()
svc.fit(trainx,trainy)
ans=svc.predict(testx)
print(f"Mean squared error:  { mean_squared_error(ans,testy, squared=False)}")

## Standard parameters with normalisation

In [ ]:
scaler=Normalizer().fit(apartments.iloc[:,1:])
cereal2=pd.DataFrame(scaler.transform(cereal.iloc[:,1:]))
X=cereal2.iloc[:,1:-1]
y=cereal["rating"]
trainx2,testx2,trainy2,testy2=train_test_split(X,y,test_size=0.3,random_state=2123)
svc2=svm.SVC()
svc2.fit(trainx2,trainy2)
ans2=svc2.predict(testx2)
print(f"Mean squared error:  { mean_squared_error(ans2,testy2, squared=False)}")


Jak można się było spodziewać standaryzacja polepszyła wynik również w drugiej bazie danych.

## Hyperparameters Random Search

In [ ]:
model_params={
    "kernel":["rbf","poly"],
    "gamma":['scale','auto'],
    "degree":[2,3,4,5,6]
    
}

In [ ]:
rs=RandomizedSearchCV(estimator = svm.SVC(),param_distributions = model_params, cv=3,n_iter=100, random_state=1613,verbose=1,scoring='neg_root_mean_squared_error')
rs.fit(trainx2,trainy2)

In [ ]:
print(f"Mean squared error:  { rs.best_score_}")
print(f"Hyperparameters:  { rs.best_params_}")

Bez większych niespodzianek random search poprawił wynik.